In [26]:
import tensorflow as tf

In [27]:
from tensorflow.examples.tutorials.mnist import input_data

In [28]:
mnist = input_data.read_data_sets("./data/", one_hot = True)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [29]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(init_random_dist)

In [30]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape = shape)
    return tf.Variable(init_bias_vals)

In [31]:
def conv2d(x, W):
    return tf.nn.conv2d(x,
                        W,
                        strides = [1, 1, 1, 1],
                        padding = "SAME")

In [32]:
def max_pool_2by2(x):
    return tf.nn.max_pool(x,
                          ksize = [1, 2, 2, 1],
                          strides = [1, 2, 2, 1],
                          padding = "SAME") 

In [33]:
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

In [34]:
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [35]:
x = tf.placeholder(tf.float32, shape = [None, 784])
y_true = tf.placeholder(tf.float32, shape = [None, 10])

In [36]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [ ]:
tf.reshape

In [37]:
convo_1 = convolutional_layer(x_image, shape = [5, 5, 1, 32])
convo_1_pooling = max_pool_2by2(convo_1)

In [38]:
convo_2 = convolutional_layer(convo_1_pooling, shape = [5, 5, 32, 64])
convo_2_pooling = max_pool_2by2(convo_2)

In [39]:
convo_2_flat = tf.reshape(convo_2_pooling, [-1, 7 * 7 * 64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [40]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob = hold_prob)

In [41]:
y_pred = normal_full_layer(full_one_dropout, 10)

In [42]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_true, 
                                                                       logits = y_pred))

In [43]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
train = optimizer.minimize(cross_entropy)

In [44]:
init = tf.global_variables_initializer()

In [45]:
steps = 2000

with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train, 
                 feed_dict = {
                     x: batch_x, 
                     y_true: batch_y, 
                     hold_prob: 0.5})
        if i%1000 == 0:
            print("Step: {}".format(i))
            matches = tf.equal(tf.arg_max(y_pred, 1), tf.argmax(y_true, 1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            print('Accuracy: ', sess.run(acc, feed_dict = {x: mnist.test.images, y_true: mnist.test.labels, hold_prob:1}))

Step: 0
Accuracy:  0.1448
Step: 1000
Accuracy:  0.985
